In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

In [2]:
torch.cuda.is_available()

False

In [2]:
class ExoplanetDataset(Dataset):
    """Exoplanet dataset."""

    def __init__(self, X, y, standardize=True, totensor=True, moving_average=1):
        self.X = X
        self.y = y
        self.standardize = standardize
        self.totensor = totensor
        self.moving_average = moving_average

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        x = self.X[idx]
        x = np.convolve(x, np.ones((self.moving_average,))/self.moving_average, mode='valid')
        
        if self.standardize:
            x = (x - np.mean(x)) / (np.std(x) + 0.01)
        
        if self.totensor:
            x = torch.tensor(x, dtype=torch.float).unsqueeze(0)

        sample = (int(self.y[idx]), x)
        return sample

In [3]:
def remove_nan(data):
    return data[~np.isnan(data).any(axis=1)]

def split_X_y(data):
    X = data[:, 0:-1]
    y = data[:, -1]- 1
    return X, y

In [8]:
### Import data ###
data_train = np.genfromtxt("../../data/kepler-labelled-time-series-data/TRAIN.csv", delimiter=',')
data_train = remove_nan(data_train)

data_valid = np.genfromtxt("../../data/kepler-labelled-time-series-data/VALIDATION.csv", delimiter=',')
data_valid = remove_nan(data_valid)

In [67]:
### Setup for training ###
X_train, y_train = split_X_y(data_train)

dataset_train = ExoplanetDataset(X_train, y_train, moving_average=100)
print("data points in training set: {}".format(len(dataset_train)))
print("training input shape: {}\n".format(dataset_train[0][1].shape))

X_val, y_val = split_X_y(data_valid)
dataset_val = ExoplanetDataset(X_val, y_val, moving_average=100)
print("data points in validation set: {}".format(len(dataset_val)))
print("validation input shape: {}\n".format(dataset_val[0][1].shape))
X_val_tens = np.asarray([dataset_val[i][1].numpy() for i in range(len(dataset_val))])
X_val_tens = torch.tensor(X_val_tens)
targets_val = np.asarray([dataset_val[i][0] for i in range(len(dataset_val))])

cuda_available = torch.cuda.is_available()

device = torch.device("cuda" if cuda_available else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda_available else {}
train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True, **kwargs)

data points in training set: 7498
training input shape: torch.Size([1, 3098])

data points in validation set: 1273
validation input shape: torch.Size([1, 3098])



In [68]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 20, kernel_size=5, stride=1)
        self.maxpool1 = nn.MaxPool1d(kernel_size=5)
        self.conv2 = nn.Conv1d(20, 50, kernel_size=5, stride=1)
        self.maxpool2 = nn.MaxPool1d(kernel_size=3)
        self.conv3 = nn.Conv1d(50, 100, kernel_size=3, stride=1)
        self.maxpool3 = nn.MaxPool1d(kernel_size=3)
        
        self.fc1 = nn.Linear(100 * 204, 500)
        self.fc2 = nn.Linear(500, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = F.relu(self.conv2(x))
        self.maxpool2(x)
        x = F.relu(self.conv3(x))
        x = self.maxpool3(x)
        x = x.view(-1, 100 * 204)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

### Model initialization ###
model = Net().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [115]:
def validate(model, X_val_tens, targets_val, threshold = 0.5):  
    preds = classify(model, X_val_tens, threshold)
    fn_ratio(preds, targets_val)
    accuracy = (preds == targets_val).sum()
   
    accuracy /= len(X_val_tens)
    print("Accuracy: {}% over {} data points. \nFN ratio: {}\n".format(
        accuracy, len(X_val_tens), fn_ratio(preds, targets_val)))
    

def train(model, device, train_loader, criterion, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (target, data) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\nLoss on train: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
            ### validate
            validate(model, X_val_tens, targets_val) 
            


### Training ###         
for epoch in range(10):
    train(model, device, train_loader, criterion, optimizer, epoch, 100)

Train Epoch: 0 [0/7498 (0%)]
Loss on train: 0.000511
Accuracy: 0.014139827179890024% over 1273 data points. 
FN ratio: 0.0

Train Epoch: 0 [3200/7498 (43%)]
Loss on train: 0.000993
Accuracy: 0.012568735271013355% over 1273 data points. 
FN ratio: 0.0

Train Epoch: 0 [6400/7498 (85%)]
Loss on train: 0.001011
Accuracy: 0.012568735271013355% over 1273 data points. 
FN ratio: 0.0

Train Epoch: 1 [0/7498 (0%)]
Loss on train: 0.000478
Accuracy: 0.015710919088766692% over 1273 data points. 
FN ratio: 0.0

Train Epoch: 1 [3200/7498 (43%)]
Loss on train: 0.000187
Accuracy: 0.014139827179890024% over 1273 data points. 
FN ratio: 0.0

Train Epoch: 1 [6400/7498 (85%)]
Loss on train: 0.000566
Accuracy: 0.012568735271013355% over 1273 data points. 
FN ratio: 0.0

Train Epoch: 2 [0/7498 (0%)]
Loss on train: 0.000803
Accuracy: 0.014139827179890024% over 1273 data points. 
FN ratio: 0.0

Train Epoch: 2 [3200/7498 (43%)]
Loss on train: 0.000436
Accuracy: 0.012568735271013355% over 1273 data points. 
FN 

KeyboardInterrupt: 

In [ ]:
def get_probabilities(model, X):
    return F.softmax(model(X), dim=1).detach().numpy()[:, 0]


def classify(model, X, threshold):
    probs = get_probabilities(model, X)
    return (probs > threshold).astype(int)


def false_negatives(preds, targets):
    is_wrong = (preds != targets).astype(int)
    is_positive = (targets == 1).astype(int)
    is_fn = ((is_wrong + is_positive) == 2).astype(int)
    return is_fn.sum()

def fn_ratio(preds, targets):
    return false_negatives(preds, targets)/len(preds)

In [109]:
preds = classify(model, X_val_tens, 0.8)
len(preds)

1273

In [5]:
a = np.array([1, 2, 3])
b = np.array([2, 1, 2])
a > b

array([False,  True,  True])

In [6]:
import sklearn

ModuleNotFoundError: No module named 'sklearn'

In [7]:
np.argmax(a)

2